In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27617677
paper_name = 'styles_andrews_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Styles2016_AllData_ForAnastasia.xlsx', sheet_name='Sheet1', skiprows=2)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5376 x 10


In [7]:
original_data.head()

,Unnamed: 0,Unnamed: 1,Fisher's Method Combined p-Values,B-Score,Fisher's Method Combined p-Values.1,B-Score.1,Fisher's Method Combined p-Values.2,B-Score.2,Fisher's Method Combined p-Values.3,B-Score.3
0,YAL002W,VPS8,9.388901,0.236979,115.184553,4.391667,2.741385,-1.40876,1.975406,-0.666667
1,YAL004W,NaN,15.657183,0.126603,16.169603,0.414583,4.699104,0.100785,5.500429,0.470833
2,YAL005C,SSA1,22.237937,-1.188797,26.292626,-1.608333,21.114808,1.57065,10.268726,2.044444
3,YAL007C,ERP2,15.361720,-0.820423,5.085368,0.300000,1.389517,-0.813889,9.223874,0.941667
4,YAL008W,FUN14,28.838803,0.010238,3.664748,1.064583,9.890748,0.950845,1.502455,-2.429167


In [8]:
original_data['orf'] = original_data['Unnamed: 0'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 1, Fisher's Method Combined p-Values, B-Score, Fisher's Method Combined p-Values.1, B-Score.1, Fisher's Method Combined p-Values.2, B-Score.2, Fisher's Method Combined p-Values.3, B-Score.3, orf]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [13]:
original_data = original_data[['B-Score','B-Score.1']].copy()

In [14]:
original_data = original_data.groupby(original_data.index).mean()

In [15]:
original_data.shape

(4268, 2)

# Prepare the final dataset

In [16]:
data = original_data.copy()

In [17]:
dataset_ids = [4928, 4929]
datasets = datasets.reindex(index=dataset_ids)

In [18]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [19]:
data.head()

dataset_id,4928,4929
data_type,value,value
orf,,
YAL002W,0.236979,4.391667
YAL004W,0.126603,0.414583
YAL005C,-1.188797,-1.608333
YAL007C,-0.820423,0.300000
YAL008W,0.010238,1.064583


## Subset to the genes currently in SGD

In [20]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [21]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [22]:
data.head()

,dataset_id,4928,4929
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.236979,4.391667
1863,YAL004W,0.126603,0.414583
4,YAL005C,-1.188797,-1.608333
5,YAL007C,-0.820423,0.300000
6,YAL008W,0.010238,1.064583


# Normalize

In [23]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [24]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [25]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [26]:
data_all.head()


,dataset_id,4928,4929,4928,4929
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.236979,4.391667,0.093631,1.814668
1863,YAL004W,0.126603,0.414583,0.040276,0.192408
4,YAL005C,-1.188797,-1.608333,-0.595579,-0.632743
5,YAL007C,-0.820423,0.300000,-0.417509,0.145670
6,YAL008W,0.010238,1.064583,-0.015974,0.457545


# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 27617677...
Inserting the new data...


100%|██████████| 2/2 [00:13<00:00,  6.88s/it]

Updating the data_modified_on field...
